In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -U sentence_transformers 

In [ ]:
! pip download -q sentence_transformers --prefer-binary --dest site_packages
! pip wheel -q sentence_transformers --wheel-dir site_packages

import numpy as np
import pandas as pd 

In [ ]:
import numpy as np 
import pandas as pd

train = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
train['target_context'] = train[['target','context']].agg(' '.join, axis=1)
train = train.drop(['id','target','context'],axis=1)
train.head()

In [ ]:
train.shape

test = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv')
test['target_context'] = test[['target','context']].agg(' '.join, axis=1)
test = test.drop(['id','target','context'],axis=1)
test.head()

from sklearn.model_selection import train_test_split

X = train.loc[:, train.columns != 'score']
y = train.score

X_train, X_test, y_train, y_test  = train_test_split(X,y,stratify=y, random_state=42, test_size=0.25)

In [ ]:
!sudo apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/sentence-transformers/all-distilroberta-v1

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')
anchor_vec = model.encode(train.anchor.tolist())
target_vec = model.encode(train.target_context.tolist())
cos_sim = []
for i in range(len(anchor_vec)):
    sim = util.cos_sim(anchor_vec[i], target_vec[i])
    cos_sim.append(sim[0][0])

In [ ]:
anchor_vec.head()

In [ ]:
cos_sim[:10]

In [ ]:
cos_sim_score = []
for i in range(len(cos_sim)):
    cs = cos_sim[i].item()
    cos_sim_score.append(cs)

In [ ]:
cos_sim_score[:10]

In [ ]:
np.around(cos_sim_score[0], 2)

In [ ]:
len(train.score)

In [ ]:
from scipy import stats

pearsonr = stats.pearsonr(train.score, cos_sim_score)
pearsonr

In [ ]:
pearsonr_score[:10]

In [ ]:
df = pd.DataFrame(data = cos_sim_score)
df.to_csv('df.csv', index=False)